# Softmax exercise

*Complete and hand in this completed worksheet (including its outputs and any supporting code outside of the worksheet) with your assignment submission. For more details see the [assignments page](http://vision.stanford.edu/teaching/cs231n/assignments.html) on the course website.*

This exercise is analogous to the SVM exercise. You will:

- implement a fully-vectorized **loss function** for the Softmax classifier
- implement the fully-vectorized expression for its **analytic gradient**
- **check your implementation** with numerical gradient
- use a validation set to **tune the learning rate and regularization** strength
- **optimize** the loss function with **SGD**
- **visualize** the final learned weights


In [1]:
import random
import numpy as np
from cs231n.data_utils import load_CIFAR10
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading extenrnal modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [2]:
def get_CIFAR10_data(num_training=49000, num_validation=1000, num_test=1000, num_dev=500):
  """
  Load the CIFAR-10 dataset from disk and perform preprocessing to prepare
  it for the linear classifier. These are the same steps as we used for the
  SVM, but condensed to a single function.  
  """
  # Load the raw CIFAR-10 data
  cifar10_dir = 'cs231n/datasets/cifar-10-batches-py'
  X_train, y_train, X_test, y_test = load_CIFAR10(cifar10_dir)
  
  # subsample the data
  mask = range(num_training, num_training + num_validation)
  X_val = X_train[mask]
  y_val = y_train[mask]
  mask = range(num_training)
  X_train = X_train[mask]
  y_train = y_train[mask]
  mask = range(num_test)
  X_test = X_test[mask]
  y_test = y_test[mask]
  mask = np.random.choice(num_training, num_dev, replace=False)
  X_dev = X_train[mask]
  y_dev = y_train[mask]
  
  # Preprocessing: reshape the image data into rows
  X_train = np.reshape(X_train, (X_train.shape[0], -1))
  X_val = np.reshape(X_val, (X_val.shape[0], -1))
  X_test = np.reshape(X_test, (X_test.shape[0], -1))
  X_dev = np.reshape(X_dev, (X_dev.shape[0], -1))
  # Normalize the data: subtract the mean image
  mean_image = np.mean(X_train, axis = 0)
  X_train -= mean_image
  X_val -= mean_image
  X_test -= mean_image
  X_dev -= mean_image
  
  # add bias dimension and transform into columns
  X_train = np.hstack([X_train, np.ones((X_train.shape[0], 1))])
  X_val = np.hstack([X_val, np.ones((X_val.shape[0], 1))])
  X_test = np.hstack([X_test, np.ones((X_test.shape[0], 1))])
  X_dev = np.hstack([X_dev, np.ones((X_dev.shape[0], 1))])
  
  return X_train, y_train, X_val, y_val, X_test, y_test, X_dev, y_dev


# Invoke the above function to get our data.
X_train, y_train, X_val, y_val, X_test, y_test, X_dev, y_dev = get_CIFAR10_data()
print 'Train data shape: ', X_train.shape
print 'Train labels shape: ', y_train.shape
print 'Validation data shape: ', X_val.shape
print 'Validation labels shape: ', y_val.shape
print 'Test data shape: ', X_test.shape
print 'Test labels shape: ', y_test.shape
print 'dev data shape: ', X_dev.shape
print 'dev labels shape: ', y_dev.shape

Train data shape:  (49000, 3073)
Train labels shape:  (49000,)
Validation data shape:  (1000, 3073)
Validation labels shape:  (1000,)
Test data shape:  (1000, 3073)
Test labels shape:  (1000,)
dev data shape:  (500, 3073)
dev labels shape:  (500,)


## Softmax Classifier

Your code for this section will all be written inside **cs231n/classifiers/softmax.py**. 


In [5]:
# First implement the naive softmax loss function with nested loops.
# Open the file cs231n/classifiers/softmax.py and implement the
# softmax_loss_naive function.

from cs231n.classifiers.softmax import softmax_loss_naive
import time

# Generate a random softmax weight matrix and use it to compute the loss.
# np.random.seed(0)  # for debugging purpose 
W = np.random.randn(3073, 10) * 0.0001
loss, grad = softmax_loss_naive(W, X_dev, y_dev, 0.0)

# As a rough sanity check, our loss should be something close to -log(0.1).
print 'loss: %f' % loss
print 'sanity check: %f' % (-np.log(0.1))

loss: 2.383803
sanity check: 2.302585


## Inline Question 1:
Why do we expect our loss to be close to -log(0.1)? Explain briefly.**

**Your answer:** starting with random weights it give a uniform  probability distrbution over all possible outputs so every output has a probabilty of ( 1/ num_of_outputs) = 0.1 


In [6]:
# Complete the implementation of softmax_loss_naive and implement a (naive)
# version of the gradient that uses nested loops.
loss, grad = softmax_loss_naive(W, X_dev, y_dev, 0.0)

# As we did for the SVM, use numeric gradient checking as a debugging tool.
# The numeric gradient should be close to the analytic gradient.
from cs231n.gradient_check import grad_check_sparse
f = lambda w: softmax_loss_naive(w, X_dev, y_dev, 0.0)[0]
grad_numerical = grad_check_sparse(f, W, grad, 10)

# similar to SVM case, do another gradient check with regularization
loss, grad = softmax_loss_naive(W, X_dev, y_dev, 1e2)
f = lambda w: softmax_loss_naive(w, X_dev, y_dev, 1e2)[0]
grad_numerical = grad_check_sparse(f, W, grad, 10)

numerical: -3.267520 analytic: -3.267520, relative error: 8.144813e-09
numerical: -0.636242 analytic: -0.636242, relative error: 1.343812e-08
numerical: -1.847735 analytic: -1.847735, relative error: 1.708735e-08
numerical: -3.413107 analytic: -3.413107, relative error: 8.021919e-09
numerical: -0.053147 analytic: -0.053147, relative error: 5.092799e-07
numerical: -2.039115 analytic: -2.039115, relative error: 1.016874e-09
numerical: 1.130399 analytic: 1.130399, relative error: 2.713217e-08
numerical: -0.230415 analytic: -0.230415, relative error: 5.316232e-08
numerical: -1.846869 analytic: -1.846869, relative error: 1.698230e-08
numerical: 3.235720 analytic: 3.235720, relative error: 2.083166e-08
numerical: 0.484146 analytic: 0.484146, relative error: 6.923334e-08
numerical: -0.286383 analytic: -0.286383, relative error: 1.769448e-07
numerical: -5.468746 analytic: -5.468746, relative error: 1.725883e-09
numerical: -0.171700 analytic: -0.171700, relative error: 5.866445e-09
numerical: 3

In [7]:
# Now that we have a naive implementation of the softmax loss function and its gradient,
# implement a vectorized version in softmax_loss_vectorized.
# The two versions should compute the same results, but the vectorized version should be
# much faster.
tic = time.time()
loss_naive, grad_naive = softmax_loss_naive(W, X_dev, y_dev, 0.00001)
toc = time.time()
print 'naive loss: %e computed in %fs' % (loss_naive, toc - tic)

from cs231n.classifiers.softmax import softmax_loss_vectorized
tic = time.time()
loss_vectorized, grad_vectorized = softmax_loss_vectorized(W, X_dev, y_dev, 0.00001)
toc = time.time()
print 'vectorized loss: %e computed in %fs' % (loss_vectorized, toc - tic)

# As we did for the SVM, we use the Frobenius norm to compare the two versions
# of the gradient.
grad_difference = np.linalg.norm(grad_naive - grad_vectorized, ord='fro')
print 'Loss difference: %f' % np.abs(loss_naive - loss_vectorized)
print 'Gradient difference: %f' % grad_difference

naive loss: 2.383803e+00 computed in 0.147631s
vectorized loss: 2.383803e+00 computed in 0.050096s
Loss difference: 0.000000
Gradient difference: 0.000000


In [9]:
# Use the validation set to tune hyperparameters (regularization strength and
# learning rate). You should experiment with different ranges for the learning
# rates and regularization strengths; if you are careful you should be able to
# get a classification accuracy of over 0.35 on the validation set.
from cs231n.classifiers import Softmax
results = {}
best_val = -1
best_softmax = None
learning_rates = [1e-7, 5e-7]
regularization_strengths = [5e4, 1e8]

################################################################################
# TODO:                                                                        #
# Use the validation set to set the learning rate and regularization strength. #
# This should be identical to the validation that you did for the SVM; save    #
# the best trained softmax classifer in best_softmax.                          #
################################################################################
for L in range(len(learning_rates)):
    for R in range(len(regularization_strengths)):
        SM_classifier = Softmax()
        SM_classifier.train(X_train,y_train,learning_rate=learning_rates[L],
                                             reg=regularization_strengths[R])
        y_pred = SM_classifier.predict(X_train)
        train_accuracy = np.mean(y_pred == y_train)
        
        y_pred = SM_classifier.predict(X_val)
        val_accuracy = np.mean(y_pred == y_val)
        results[(learning_rates[L],regularization_strengths[R])]=(train_accuracy,val_accuracy)
        if val_accuracy > best_val:
            best_val = val_accuracy
            best_softmax = SM_classifier
################################################################################
#                              END OF YOUR CODE                                #
################################################################################
    
# Print out results.
for lr, reg in sorted(results):
    train_accuracy, val_accuracy = results[(lr, reg)]
    print 'lr %e reg %e train accuracy: %f val accuracy: %f' % (
                lr, reg, train_accuracy, val_accuracy)
    
print 'best validation accuracy achieved during cross-validation: %f' % best_val

cs231n/classifiers/softmax.py:79: RuntimeWarning: divide by zero encountered in log
  loss_vec = -1 * np.log (probabilites_of_classes[range(N),y])
cs231n/classifiers/softmax.py:89: RuntimeWarning: overflow encountered in double_scalars
  loss += 0.5 * reg * np.sum(W * W)
cs231n/classifiers/softmax.py:89: RuntimeWarning: overflow encountered in multiply
  loss += 0.5 * reg * np.sum(W * W)


lr 1.000000e-07 reg 5.000000e+04 train accuracy: 0.147939 val accuracy: 0.140000
lr 1.000000e-07 reg 1.000000e+08 train accuracy: 0.077367 val accuracy: 0.075000
lr 5.000000e-07 reg 5.000000e+04 train accuracy: 0.303918 val accuracy: 0.304000
lr 5.000000e-07 reg 1.000000e+08 train accuracy: 0.094204 val accuracy: 0.085000
best validation accuracy achieved during cross-validation: 0.304000


In [10]:
best_softmax.train(X_train,y_train,learning_rate=5e-7,reg=5e4,num_iters=10000,verbose=True)
# evaluate on test set
# Evaluate the best softmax on test set
y_test_pred = best_softmax.predict(X_test)
test_accuracy = np.mean(y_test == y_test_pred)
print 'softmax on raw pixels final test set accuracy: %f' % (test_accuracy, )

iteration 0 / 10000: loss 6.912756
iteration 100 / 10000: loss 2.141964
iteration 200 / 10000: loss 2.122091
iteration 300 / 10000: loss 2.098252
iteration 400 / 10000: loss 2.111604
iteration 500 / 10000: loss 2.116483
iteration 600 / 10000: loss 2.072360
iteration 700 / 10000: loss 2.173260
iteration 800 / 10000: loss 2.085107
iteration 900 / 10000: loss 2.098072
iteration 1000 / 10000: loss 2.056223
iteration 1100 / 10000: loss 2.103981
iteration 1200 / 10000: loss 2.055843
iteration 1300 / 10000: loss 2.061591
iteration 1400 / 10000: loss 2.026963
iteration 1500 / 10000: loss 2.114552
iteration 1600 / 10000: loss 2.083350
iteration 1700 / 10000: loss 2.071725
iteration 1800 / 10000: loss 2.104909
iteration 1900 / 10000: loss 2.118530
iteration 2000 / 10000: loss 2.058000
iteration 2100 / 10000: loss 2.051917
iteration 2200 / 10000: loss 2.119611
iteration 2300 / 10000: loss 2.153112
iteration 2400 / 10000: loss 2.067575
iteration 2500 / 10000: loss 2.096941
iteration 2600 / 10000: 

In [11]:
# Visualize the learned weights for each class
w = best_softmax.W[:-1,:] # strip out the bias
w = w.reshape(32, 32, 3, 10)

w_min, w_max = np.min(w), np.max(w)

classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
for i in xrange(10):
  plt.subplot(2, 5, i + 1)
  
  # Rescale the weights to be between 0 and 255
  wimg = 255.0 * (w[:, :, :, i].squeeze() - w_min) / (w_max - w_min)
  plt.imshow(wimg.astype('uint8'))
  plt.axis('off')
  plt.title(classes[i])